In [159]:
import pandas as pd
import numpy as np
df = pd.read_csv('F:/DL/HAKATON/int20h-ds-test-dataset.csv')

In [160]:
df

,userid,user_state,event_name,event_attributes,event_created_date,event_platform,device_manufacture,device_model
0,c95c777785faec8dd910d019d7278ebe,CA,Add Vehicle Success,"{""Make"":""Dodge"",""Model"":""Caravan"",""Color"":""Whi...",2022-01-16 17:03:04,android,samsung,SM-N975U
1,c95c777785faec8dd910d019d7278ebe,CA,Add Vehicle Break,{},2022-01-16 17:07:47,android,samsung,SM-N975U
2,f344be2d9a042b7444f3cc5279e38ef1,FL,Calculator View,{},2022-01-16 17:16:25,android,samsung,SM-G973U1
3,c95c777785faec8dd910d019d7278ebe,CA,Add Payment Method Success,"{""Payment Method"":""Credit"",""Tokenized Pay"":""""}",2022-01-16 17:24:22,android,samsung,SM-N975U
4,e331ed81422d8fba55520a43a872e701,IL,Sign Up Success,"{""Method"":""Apple""}",2022-01-16 17:34:51,ios,Apple,"iPhone12,1"
...,...,...,...,...,...,...,...,...
23352,679eba26c4e75e0afb178360becfa21b,CA,Add Payment Method Success,"{""Payment Method"":""Credit"",""Tokenized Pay"":"""",...",2022-04-16 20:49:24,android,Google,Pixel 3a
23353,679eba26c4e75e0afb178360becfa21b,CA,Account Setup Profile Skip,"{""Screen"":""Address""}",2022-04-16 20:50:05,android,Google,Pixel 3a
23354,679eba26c4e75e0afb178360becfa21b,CA,Account Setup Profile Skip,"{""Screen"":""Phone Number""}",2022-04-16 20:50:10,android,Google,Pixel 3a
23355,679eba26c4e75e0afb178360becfa21b,CA,Chat Conversation Opened,"{""From"":""Dashboard"",""Transaction type"":""""}",2022-04-16 20:50:31,android,Google,Pixel 3a


In [161]:
gone_users = df[df['event_name'] == 'Subscription Premium Cancel'].userid.unique().tolist()

In [162]:
df['gone_user'] = df['userid'].isin(gone_users)

In [163]:
count_events = df[['userid','user_state', 'event_name', 'gone_user']]

In [183]:
count_events2 = df[['userid', 'event_name', 'gone_user']]

In [184]:
df3 = count_events2.groupby(['userid', 'event_name']).count().unstack(level=1,fill_value=0)['gone_user']

In [186]:
df3['gone_user'] = df3.index.isin(gone_users)

In [191]:
df4 = df3.groupby(['gone_user']).mean()

In [202]:
df5 = df4.T

In [206]:
df5

gone_user,False,True
event_name,,
Account History Transaction Details,0.323199,2.175299
Account Setup Profile Skip,0.134387,0.223108
Account Setup Skip,0.064457,0.039841
Add Payment Method Failed,0.093341,0.107570
Add Payment Method Success,0.238978,1.003984
Add Vehicle Break,0.131043,0.219124
Add Vehicle Failed,0.006081,0.003984
Add Vehicle Success,0.514442,0.920319
Calculator Used,0.030404,0.079681


In [205]:
df5.sort_values('False')

KeyError: 'False'

In [164]:
df1 = count_events.groupby(['user_state','userid', 'event_name']).count().unstack(level=2,fill_value=0)['gone_user']

In [166]:
df1.reset_index(level = 'user_state', inplace= True)

In [168]:
df1['gone_user'] = df1.index.isin(gone_users)

In [174]:
df2 = df1.groupby(['user_state','gone_user']).mean()

In [187]:
df2

event_name            Account History Transaction Details  \
user_state gone_user                                        
AK         False                                 0.000000   
AL         False                                 0.200000   
           True                                  1.000000   
AR         False                                 0.000000   
           True                                  1.500000   
...                                                   ...   
WA         False                                 0.000000   
WI         False                                 0.117647   
           True                                  0.000000   
WV         False                                 0.000000   
           True                                  3.000000   

event_name            Account Setup Profile Skip  Account Setup Skip  \
user_state gone_user                                                   
AK         False                             0.0            0.000000   
AL         False                             0.0            0.000000   
           True                              0.0            0.000000   
AR         False                             0.0            0.100000   
           True                              0.0            0.000000   
...                                          ...                 ...   
WA         False                             0.0            0.000000   
WI         False                             0.0            0.058824   
           True                              0.0            0.000000   
WV         False                             0.0            0.000000   
           True                              0.0            0.000000   

event_name            Add Payment Method Failed  Add Payment Method Success  \
user_state gone_user                                                          
AK         False                       0.000000                    0.000000   
AL         False                       0.200000                    0.200000   
           True                        0.000000                    1.000000   
AR         False                       0.000000                    0.000000   
           True                        0.000000                    1.000000   
...                                         ...                         ...   
WA         False                       0.000000                    0.000000   
WI         False                       0.176471                    0.294118   
           True                        0.000000                    0.000000   
WV         False                       0.166667                    0.000000   
           True                        0.000000                    1.000000   

event_name            Add Vehicle Break  Add Vehicle Failed  \
user_state gone_user                                          
AK         False               0.000000                 0.0   
AL         False               0.000000                 0.0   
           True                0.000000                 0.0   
AR         False               0.100000                 0.0   
           True                0.000000                 0.0   
...                                 ...                 ...   
WA         False               0.000000                 0.0   
WI         False               0.117647                 0.0   
           True                0.000000                 0.0   
WV         False               0.166667                 0.0   
           True                0.000000                 0.0   

event_name            Add Vehicle Success  Calculator Used  Calculator View  \
user_state gone_user                                                          
AK         False                 0.000000         0.000000         0.000000   
AL         False                 0.400000         0.000000         0.000000   
           True                  1.000000         0.000000         0.000000   
AR         False          